## Demo Scenario
We are going to be analyzing some rental data

In [ ]:
import pandas as pd
# load data
contacts = pd.read_csv("/Users/yifanwu/Dev/midas/notebooks/rental/contacts.csv")
listings = pd.read_csv("/Users/yifanwu/Dev/midas/notebooks/rental/listings.csv")
users = pd.read_csv("/Users/yifanwu/Dev/midas/notebooks/rental/users.csv")

## Basic Profiling and Visualizations
When loading data into Midas, we show a basic profiler on the side.

In [ ]:
from midas import Midas
m = Midas()

In [ ]:
contacts_df = m.register_df("contacts_df", contacts)

In [ ]:
listings_df = m.register_df("listings_df", listings)
users_df = m.register_df("users_df", users)

In [ ]:
# now let's say that there is an interesting chart the user wants to look at
contact_channel_first_df = contacts_df.project([contacts_df.contact_channel_first]).assign("contact_channel_first_df")


In [ ]:
m_interactions_df = contacts_df.project([contacts_df.m_interactions]).assign("m_interactions_df")


## Access Selections Stream

In [ ]:
contact_channel_stream = contact_channel_first_df.get_stream()
contact_channel_stream

In [ ]:
contact_channel_stream.current

In [ ]:
contacts_df.project([contacts_df.ts_booking_at]).pandas_value

In [ ]:
room_type_df = listings_df.project([listings_df.room_type]).assign("room_type_df")

In [ ]:
# note the code below does not yet work!

In [ ]:
total_reviews_df = listings_df.project([listings_df.total_reviews]).assign("total_reviews_df")


## Add Callbacks to Streams

In [ ]:
def transform(predicate):
    m.ui_comm.send_debug_msg(f"Transform {predicate}")
    new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
    new_mdf.project([new_mdf.m_guests]).assign("m_guests")

contact_channel_stream.add_callback(transform)

In [ ]:
new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
m_guests = new_mdf.project([new_mdf.m_guests]).assign("m_guests")
m_guests.pandas_value

In [ ]:
contact_channel_stream.ref_to_predicate_list

In [ ]:
m.event_loop.tick_funcs['contact_channel_first_df'] = []

In [ ]:
p = contact_channel_stream.current
m.event_loop.tick('contact_channel_first_df', p)

In [ ]:
predicate = contact_channel_stream.current
new_mdf = contacts_df.apply_selection(contact_channel_stream.current)
m_guests = new_mdf.project([new_mdf.m_guests]).assign("m_guests") 


In [ ]:
m_guests.pan

In [ ]:
m_guests_df = new_mdf.project([new_mdf.m_guests]).assign("m_guests")


In [ ]:
p = contacts_df.pandas_value

p[p["contact_channel_first"].isin(['contact_me', 'book_it'])]


In [ ]:
contact_channel_first_df.pandas_value

In [ ]:
new_mdf.pandas_value

In [ ]:
new_mdf.ops.predicate.operation

## Linking Static Charts

## UI Shortcuts